In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn.functional as F

from implcarv.utils.plotly import PlotlyVisualizer

visualizer = PlotlyVisualizer()

meta_data = np.load("../results/Bear_cub_full_IP/meta_data.npz")

rest_bones_tail = torch.from_numpy(meta_data["rest_loc_tail"]).float()
rest_bones_head = torch.from_numpy(meta_data["rest_loc_head"]).float()

verts = torch.from_numpy(meta_data["verts"]).float()
rest_verts = torch.from_numpy(meta_data["rest_verts"]).float()
tfs = torch.from_numpy(meta_data["pose_matrix"]).float()
bones_tail = torch.einsum("bnij,nj->bni", tfs[..., 0:3, :], F.pad(rest_bones_tail, pad=[0, 1], value=1))
bones_head = torch.einsum("bnij,nj->bni", tfs[..., 0:3, :], F.pad(rest_bones_head, pad=[0, 1], value=1))
print ("verts", verts.shape, "tfs", tfs.shape)

frame_id = 10
visualizer.create_figure("figure")
# visualizer.add_pointcloud("verts", verts[frame_id], color=[0.5, 0.5, 0.5])
# visualizer.add_pointcloud("bones_head", bones_head[frame_id], color=[1, 0, 0])
# visualizer.add_pointcloud("bones_tail", bones_tail[frame_id], color=[1, 1, 0])
visualizer.add_pointcloud("verts", rest_verts, color=[0.5, 0.5, 0.5])
visualizer.add_pointcloud("bones_head", rest_bones_head, color=[1, 0, 0])
visualizer.add_pointcloud("bones_tail", rest_bones_tail, color=[1, 1, 0])
visualizer.show()


In [ ]:
names = meta_data["names"]
stds = tfs.std(dim=0).amax(dim=[-1, -2])

for name, std in zip(names, stds):
    print (name, std)

In [ ]:
import imageio

camera_id = "cam_002"

image = imageio.imread("../results/Bear_cub_full_IP/image/%s/%08d.png" % (camera_id, frame_id))
print (image.shape)
image = image[:, :, 0:3]

with open("../results/Bear_cub_full_IP/camera.json", "r") as fp:
    cameras = json.load(fp)
    camera = cameras[camera_id]
    K = torch.tensor(camera["intrin"]).float()
    w2c = torch.tensor(camera["extrin"]).float()
    print (K)
    print (w2c)


In [ ]:
import cv2

bones_head_proj = torch.einsum(
    "ij,jk,nk->ni", K, w2c[:3, :], F.pad(bones_head[frame_id], pad=[0, 1], value=1)
)
bones_head_proj = bones_head_proj[..., 0:2] / bones_head_proj[..., 2:3]

canvas = image.copy()
for x, y in bones_head_proj:
    cv2.circle(canvas, 
               (int(x), int(y)),
               radius=5,
               color=(255, 0, 0),
               thickness=-1)

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 10)

plt.imshow(canvas)


In [ ]:
tt = np.pad(meta_data["rest_loc_head"], pad_width=[(0, 0), (0, 1)], constant_values=1)
tt[0]


In [ ]:
meta_data["rest_loc_head"].shape

In [ ]:
print (bones_head[:, meta_data["names"].tolist().index("Base_spine")].mean())
print (bones_head[:, meta_data["names"].tolist().index("spine_2")].mean())
print (bones_head[:, meta_data["names"].tolist().index("spine_3")].mean())

print (tfs[:, meta_data["names"].tolist().index("Base_spine")].mean())
print (tfs[:, meta_data["names"].tolist().index("spine_2")].mean())
print (tfs[:, meta_data["names"].tolist().index("spine_3")].mean())



In [ ]:
print (bones_head[:, meta_data["names"].tolist().index("Helper_foot_f.R")].mean())
print (bones_head[:, meta_data["names"].tolist().index("foot_f.R")].mean())

print (tfs[:, meta_data["names"].tolist().index("Helper_foot_f.R")].mean())
print (tfs[:, meta_data["names"].tolist().index("foot_f.R")].mean())

